# Training, testing and evaluating models for data


In [70]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import pandas as pd
import numpy as np
from taxipred.utils.constants import get_clean_data
from Cleaning_functions import split_features_target
from model_functions import hyper_optimize, train_evaluate, tune_model

df = get_clean_data()
df.head()

,Trip_Distance_km,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Time_of_Day_Afternoon,Day_of_Week_Weekday,Traffic_Conditions_High,Weather_Rain,Weather_Snow,Trip_Price
0,19.350000,3.560000,0.80,0.32,53.82,False,True,False,False,False,36.2624
1,47.590000,3.502989,0.62,0.43,40.57,True,True,True,False,False,53.6163
2,36.870000,2.700000,1.21,0.15,37.27,False,False,True,False,False,52.9032
3,30.330000,3.480000,0.51,0.15,116.81,False,True,False,False,False,36.4698
4,27.070547,2.930000,0.63,0.32,22.64,False,True,True,False,False,15.6180


In [71]:
df_numeric, df_categorical, df_target = split_features_target(df)

### Splitting the data into target and features

In [72]:
# merging features into one dataframe
df_features = pd.concat([df_numeric, df_categorical], axis=1)

X, y = df_features, df_target
X.head()

,Trip_Distance_km,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes
0,19.350000,3.560000,0.80,0.32,53.82
1,47.590000,3.502989,0.62,0.43,40.57
2,36.870000,2.700000,1.21,0.15,37.27
3,30.330000,3.480000,0.51,0.15,116.81
4,27.070547,2.930000,0.63,0.32,22.64


### train|test split

In [73]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()


X_train.shape, X_test.shape

((800, 5), (200, 5))

### method for prediction models 

https://www.geeksforgeeks.org/machine-learning/hyperparameter-tuning-in-linear-regression/

In [74]:

# creating a dict for different estimators to evaluate
models = {
  #"linear": {"model": LinearRegression(), "scale": True},
  #"ridge": {"model": Ridge(), "scale": True},
  #"lasso":{"model": Lasso(), "scale": True},
  #"knn": {"model": KNeighborsRegressor(), "scale": True},
  #"svr": {"model": SVR(), "scale": True},
  #"mlp": {"model": MLPRegressor(max_iter=200, random_state=42), "scale": True},
  "xgb": {"model": XGBRegressor(random_state=42), "scale": True},
  #"rf": {"model": RandomForestRegressor(random_state=42), "scale": False}
}

In [75]:
df_results = hyper_optimize(X_train, y_train, models)

In [76]:

df_results = df_results.sort_values(by="Best R2", ascending=False)
    

In [77]:
df_results

,Model,Best R2,Best Params
0,xgb,0.869875,"{'model__learning_rate': 0.05, 'model__max_dep..."


### tuning the model for with cross-validation for omptimal hyperparameters

In [78]:

tuned_model = tune_model(df_results, "xgb", models)

# train/eval
model, metrics = train_evaluate(X_train, y_train, X_test, y_test, tuned_model)

print(metrics)


{'MAE': 4.97, 'MSE': 54.85, 'RMSE': np.float64(7.41), 'R2': 0.91}


### exporting data using joblib

In [80]:
# model, metrics = train_evaluate(X_train, y_train, X_test, y_test, model_type="random_forest", model_path="random_forest.joblib")
# loaded_model = joblib.load("random_forest.joblib")
# loaded_model